# Georeferencing Tutorial

To georeference an aerial photo is to establish a relationship between the internal coordinate system of the photo, and geographic coordinate system. As a result, for each pixel on the aerial photo, its latitude and longitude are known. In this tutorial, you will learn the georeferencing algorithm and how to apply it to aerial photos.

![georeferencing](assets/georeferencing.png)

## Input Data

Georeferencing algorithm needs input data to run - but what is this data and where does it come from? To georeference an aerial photo, we need to know the geographic location (latitude, longitude, and altitude) of the UAV when the photo was captured (in an ideal world, we would use the geographic location of the camera, but the error from using UAV's location is so small that it's negligible) and the orientation of the gimbal on which the camera is mounted. One of the main purposes of the on-board computer (OBC) is to orchestrate the collection of that data - the OBC triggers image capture, and immediately reads GPS and gimbal control board values. After the image is captured, the OBC adds geographic location and gimbal orientation information to the metadata of the image (more on this later), and then sends this image to the Ground Control Station (GCS). 

### Sample Image

Below is an image captured at the Surrey field - we will use it as an example throughout this tutorial. 

![sample-img](assets/sample-img.jpg)

Before we can run the georeferencing algorithm on the photo, we should extract the metadata with geographic location and gimbal orientation information. Metadata is stored in well-documented [Exif](https://en.wikipedia.org/wiki/Exif) tags, and we access it using `pyexiftool` [library](https://github.com/smarnach/pyexiftool). This library allows us to use a command-line `ExifTool` [utility](https://www.sno.phy.queensu.ca/~phil/exiftool/) from within Python programs.

In [13]:
import exiftool

exiftool_proc = exiftool.ExifTool() # create an instance of ExifTool process
exiftool_proc.start()
metadata = exiftool_proc.get_metadata("assets/georeferencing1.jpg")

WindowsError: [Error 2] The system cannot find the file specified

## Algorithm

We begin with a set of coordinates in the geodetic frame: latitude, longitude, and altitude of the UAV when the photo is taken.

In [10]:
import numpy as np
from math import radians, degrees

## References